In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from functools import partial
from tqdm.notebook import tqdm
from IPython.display import HTML
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from transformers_gradients import (
    text_classification,
    visualise_explanations_as_html,
    SmoothGradConfing,
    NoiseGradConfig,
    NoiseGradPlusPlusConfig,
)


tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

Metal device set to: Apple M1 Pro


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
x = ["Like four times a year I rediscover Björk and listen to her full discography"]
y = [1]

In [8]:
a_batch = [
    f(model, x, y, tokenizer=tokenizer)[0]
    for f in tqdm(
        [
            text_classification.gradient_norm,
            text_classification.gradient_x_input,
            text_classification.integrated_gradients,
            partial(
                text_classification.smooth_grad,
                config=SmoothGradConfing(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.noise_grad,
                config=NoiseGradConfig(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.noise_grad_plus_plus,
                config=NoiseGradPlusPlusConfig(explain_fn="GradNorm"),
            ),
            partial(
                text_classification.smooth_grad,
                config=SmoothGradConfing(explain_fn="GradXInput"),
            ),
            partial(
                text_classification.noise_grad,
                config=NoiseGradConfig(explain_fn="GradXInput"),
            ),
            partial(
                text_classification.noise_grad_plus_plus,
                config=NoiseGradPlusPlusConfig(explain_fn="GradXInput"),
            ),
            text_classification.smooth_grad,
            text_classification.noise_grad,
            text_classification.noise_grad_plus_plus,
        ]
    )
]

  0%|          | 0/12 [00:00<?, ?it/s]

2023-04-20 21:58:37,746:[polymorphic_function.py:156->called_with_tracing()]:WARNING: 5 out of the last 5 calls to <function pfor.<locals>.f at 0x39930b0a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2023-04-20 21:58:37,948:[polymorphic_function.py:156->called_with_tracing()]:WARNING: 6 out of the last 6 calls to <function pfor.<locals>.f at 0x399309120> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


In [9]:
html = visualise_explanations_as_html(
    a_batch,
    labels=[
        "Gradient Norm",
        "Gradient X Input",
        "Integrated Gradients",
        "Smooth Grad + Gradient Norm",
        "Noise Grad + Gradient Norm",
        "NoiseGrad++ + Gradient Norm",
        "Smooth Grad + Gradient X Input",
        "Noise Grad + Gradient X Input",
        "NoiseGrad++ + Gradient X Input",
        "Smooth Grad + Integrated Gradients",
        "Noise Grad + Integrated Gradients",
        "NoiseGrad++ + Integrated Gradients",
    ],
)

HTML(html)